A first order liquid phase reaction where component A reacts to form B using a porous catalyst is studied in a packed bed reactor. The following is known about the fixed bed setup:

<img src="Tut5.3 data.JPG" width="450" />

You can use the Thoenes-Kramer correlation to estimate the liquid solid mass
transfer coefficient (if required)

In [35]:
from numpy import *
from scipy.optimize import fsolve
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

In [36]:
W       =          0.9 #kg
dp      =       2/1000 #m
R       =         dp/2 #m
rhobed  =         1300 #kg/m3
De      =       1.8e-9 #m2/s
Q       = 60/3600/1000 #m3/s
X       =        0.895
D       =         0.04 #m reactor diameter
kiprime =         5e-4 #m3/(kg.s) 
Dab     =         3e-9 #m2/s
mu      =        0.001 #kg/(m.s)
epsilon =          0.4 #porosity
rholiq  =         1000 #kg/m3

a) What is the catalyst effectiveness factor? ($\eta = 0.117$)

$$
\begin{align}
\frac{dF_A}{dW}&=-r_{A}^{'}\\
\frac{dF_A}{dW}&=\eta k_{i}^{'}C_A\\
QC_{Ao}\frac{dx}{dW}&=\eta k_{i}^{'}C_{Ao}(1-x)\\
\int^{x=0.895}_{x=0}\frac{dx}{1-x}&=\frac{\eta k_{i}^{'}W}{Q}\\
-\ln(1-x)\big\rvert^{0.895}_0 &=\frac{\eta k_{i}^{'}W}{Q}\\
\eta k_{i}^{'} &=-\frac{Q\ln(1-x)\big\rvert^{0.895}_0}{W} \\
\eta &= -ln(1 - x)\dfrac{Q}{k_{i}^{'}.W}  
\end{align}
$$

Since mass transfer is NOT negligible, the above equation can NOT be used.

**THIELE MODULUS**

\begin{align}
\phi_{n}^{2}&=\frac{k_i'\rho_cR^2C_{As}^{n-1}}{D_e}\\
\phi_{1}&=\left(\frac{k_i'\rho_bR^2}{D_e(1-\epsilon)}\right)^{0.5}\\
\end{align}

In [37]:
a   = kiprime*rhobed*R**2
b   = De*(1 - epsilon)
phi = (a/b)**0.5
print(str(phi))

24.532669073132908


For a first order reaction

\begin{align}
\eta&=\frac{3}{\phi_1^2}\left(\phi_1\coth\phi_1-1\right)
\end{align}

In [38]:
eta = 3*((phi/tanh(phi)) - 1)/(phi**2)
print(str(eta))

0.11730130430300095


**b) Does liquid-solid mass transfer play a role?** Answer this question without the use of a correlation. ($x_{ideal} = 0.96 > x_{real}$ , Yes, Mass transfer plays a role)

If only internal mass transfer is considered $k_{eff}'=\eta k_i'$. If this conversion is lower than what is actually obtained, it implies that the rate is influenced by external mass transfer and $$\frac{1}{k_{eff}'}=\frac{1}{k_ca_m}+\frac{1}{\eta k_i'}$$

$$
k_{eff}=\frac{\eta k_i'k_ca_m}{\eta k_i'+k_ca_m}
$$
and from above derivation:
\begin{align}
-ln(1 - x) &= \eta k_{i}^{'}\dfrac{W}{Q} \\
-ln(1 - x) &= k_{eff}^{'}\dfrac{W}{Q}
\end{align}
Now solve for conversion:

In [39]:
c = -eta*kiprime*W/Q
d = exp(c)
Xideal = 1 - d
print(str(Xideal))

0.9578758983427543


Therefore, since $x_{ideal}=0.958 > x_{actual}=0.895$, mass transfer does indeed affect the rate.

**c) The reactor diameter is changed to 2 cm while still using the same mass of catalyst (i.e a longer reactor), what is the new conversion?** ($X_{new} = 0.928$ with  $k_{eff}^{'} = 4.877 \times 10^{−5}$)

Answer: The Thoenes Kramer correlation can be used in this question. The first thing to notice, all other catalysts specifications kept constant, is that reducing the reactor diameter will increase the linear velocity. From this, we can calculate the new $k_{c}$ value, and this unlocks the remainder of the solution.

**Step 1:** 

We use the Thoenes Kramer correlation to calculate the new $k_{c}$ value.

\begin{align}
\left[\frac{k_cd_p}{D_{AB}}\left(\frac{\epsilon}{1-\epsilon}\right)\frac{1}{\gamma}\right] &= \left[\frac{Ud_p\rho_L}{\mu(1-\epsilon)\gamma}\right]^{1/2}\left(\frac{\mu}{\rho D_{AB}}\right)^{1/3}\\
\end{align}

In [40]:
D1       = D
Aacross1 = 0.25*pi*(D1)**2
U1       = Q/Aacross1
Asurface = pi*dp**2
gamma    = Asurface/(pi*dp**2)

In [41]:
a = (U1*dp*rholiq/(mu*(1 - epsilon)*gamma))**0.5
b = (mu/(rholiq*Dab))**(1/3)
c = dp*epsilon/(Dab*(1 - epsilon)*gamma)

In [42]:
kc1 = a*b/c
print(str(kc1))

0.00010372917297430709


**Step 2:**

Dividing the new Thoenes Kramer correlation (containing the new $k_{c}$ and new $U$) by the first with original $k_{c}$ and $U$ results in:

\begin{equation}
\frac{k_{c2}}{k_{c1}}=\left[\frac{U_2}{U_1}\right]^{1/2}\\
\end{equation}

In [43]:
D2       = 0.02 #m new reactor diameter
Aacross2 = 0.25*pi*(D2)**2
U2       = Q/Aacross2

In [44]:
kc2 = kc1*(U2/U1)**0.5
print(str(kc2))

0.00020745834594861418


**Step 3:**

Calculate $a_{m}$ from:
\begin{equation}
a_{m} = \frac{6(1-\epsilon)}{d_p\rho_b}
\end{equation}

And $k_{eff}^{'}$ from:
\begin{equation}
k_{eff}^{'}=\frac{\eta k_{i}^{'}k_{c}a_{m}}{\eta k_{i}^{'} + k_{c}a_{m}}
\end{equation}

In [45]:
am = 6*(1 - epsilon)/(dp*rhobed)
print(str(am))

1.3846153846153844


In [46]:
keffprime = eta*kiprime*kc2*am/(eta*kiprime + kc2*am)
print(str(keffprime))

4.870589257185835e-05


**Step 4:**

Now we can use the mole balance equation derived above:

\begin{equation}
-ln(1 - X) = k_{eff}^{'}\dfrac{W}{Q}
\end{equation}

In [47]:
d  = keffprime*W/Q
e  = exp(-d)
X2 = 1 - e
print('Xnew= '+str(X2), 'Keff= '+str(keffprime))

Xnew= 0.9279300568512846 Keff= 4.870589257185835e-05


**d) Calculate the conversion if the 4 cm reactor is used with 1.5 mm catalyst (same total mass of catalyst)** ($X_{new}= 0.95$, $a_{m} = 1.846$ and $ k_{eff}^{'} = 5.72 \times 10^{−5}~m^{3}/(kg.s)$)

In [48]:
dp3 = 1.5/1000 #m 

Extrapolating the methodology used in the previous question, a new $k_{c}$ value can be obtained by now leaving the linear velocity unchanged and correlating the change in particle diameter. The new proportionality derived from Theone Kramer:

\begin{align}
\frac{k_{c3}d_{p3}}{k_{c1}d_{p1}}&=\left[\frac{d_{p3}}{d_{p1}}\right]^{1/2}\\
\frac{k_{c3}}{k_{c1}}&=\left[\frac{d_{p1}}{d_{p3}}\right]^{1/2}
\end{align}

In [49]:
dp1 = dp
kc3 = kc1*(dp1/dp3)**0.5
print(str(kc3))

0.0001197761318790669


The other affected variables are $a_{m}$, $\phi$ and $\eta$. From these a new $k_{eff}^{'}$ can be obtained to calculate the new conversion

In [52]:
am3 = 6*(1 - epsilon)/(dp3*rhobed)
print(str(am3))

1.846153846153846


From the Thiele modulus:
\begin{align}
\phi_n^2&=\frac{k_i'\rho_cR^2C_{As}^{n-1}}{D_e}\\
&=\frac{\frac{k_{app}}{\eta}\rho_cR^2C_{As}^{n-1}}{D_e}\\
\phi_1&=\left(\frac{k_{i}^{'}\rho_{bed}R^{2}}{D_{e}(1 - \epsilon) }\right)^{0.5}
\end{align}

In [60]:
R3   = dp3/2
phi3 = (kiprime*rhobed*(R3**2)/(De*(1 - epsilon)))**0.5 
print(str(phi3))

18.39950180484968


\begin{equation}
\eta=\frac{3}{\phi_1^2}\left(\phi_1\coth\phi_1-1\right)\\
\end{equation}

In [63]:
eta3 = 3*((phi3/tanh(phi3))-1)/(phi3**2)
print(str(eta3))

0.15418635445528334


\begin{equation}
k_{eff}'=\frac{\eta k'_ik_ca_m}{\eta k'_i + k_ca_m}\\
\end{equation}

In [65]:
keff3 = eta3*kiprime*kc3*am3/(eta*kiprime + kc3*am3)
print(str(keff3))

5.71636252763895e-05
